In [2]:
!pip install --upgrade google-cloud-aiplatform==1.49.0 langchain==0.1.16 langchain-google-vertexai==1.0.2 pydantic

  Using cached google_cloud_aiplatform-1.49.0-py2.py3-none-any.whl.metadata (30 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
Using cached google_cloud_aiplatform-1.49.0-py2.py3-none-any.whl (4.9 MB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.49.dev20240422+vertex.rag
    Uninstalling google-cloud-aiplatform-1.49.dev20240422+vertex.rag:
      Successfully uninstalled google-cloud-aiplatform-1.49.dev20240422+vertex.rag


In [1]:
PROJECT_ID = "andrewcooley-test-project"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://andrewcooley-reasoning-engine"

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [2]:
import requests
from vertexai.preview import reasoning_engines

In [3]:
model = "gemini-1.5-pro-preview-0514"

In [4]:
def get_prompt_outputs_from_language(language: str) -> str:
    """Query a BigQuery table using a user input for the WHERE clause to filter on the 'language' field in the table.

    Args:
        language (str): the language to use for the WHERE clause filter.

    Returns:
        str: the count of records with that language.
    """
    import json
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    client = bigquery.Client()  # Automatically uses your local authentication setup

    # Define your query parameters
    project_id = 'andrewcooley-test-project'
    dataset_id = 'genai'
    table_id = 'writing_composer_prompts'

    query = f"""
        SELECT COUNT(*) as count
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE language LIKE '%{language}%'
    """

    # Run the query
    query_job = client.query(query)  

    # Fetch results
    results = query_job.result()

    for row in results:
        records = row["count"]
    
    records = str(records)

    return records

def get_prompt_outputs_from_expert_rating(expert_rating: str) -> str:
    """Query a BigQuery table using a user input for the WHERE clause to filter on the 'expert rating' field in the table.

    Args:
        expert rating (str): the wildcard matched text to use for the WHERE clause filter.

    Returns:
        str: outputs that match the filter set in the WHERE clause.
    """
    import json
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    client = bigquery.Client()  # Automatically uses your local authentication setup

    # Define your query parameters
    project_id = 'andrewcooley-test-project'
    dataset_id = 'genai'
    table_id = 'writing_composer_prompts'

    query = f"""
        SELECT expert_rating, output
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE expert_rating LIKE '%{expert_rating}%'
    """

    # Run the query
    query_job = client.query(query)  

    # Fetch results
    results = query_job.result()

    for row in results:
        records = row["output"]
    
    records = str(records)

    return records

In [5]:
# get_prompt_outputs_from_language(language="English")
get_prompt_outputs_from_expert_rating(expert_rating="Excellent")

'Mi tarea es imprimir el contenido rodeado por [entrada] y EOS.\n[entrada]'

In [6]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_prompt_outputs_from_expert_rating],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [7]:
agent.query(input="Were there any outputs that had a grade of '5' in the expert rating? Can you share what the output was for that rating?")

{'input': "Were there any outputs that had a grade of '5' in the expert rating? Can you share what the output was for that rating?",
 'output': "The output for the grade '5' was: \n```\nMi tarea es imprimir el contenido rodeado por [entrada] y EOS.\n[entrada]\n```",
 'intermediate_steps': [[{'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'agent', 'AgentActionMessageLog'],
    'kwargs': {'tool': 'get_prompt_outputs_from_expert_rating',
     'tool_input': {'expert_rating': '5'},
     'log': "\nInvoking: `get_prompt_outputs_from_expert_rating` with `{'expert_rating': '5'}`\n\n\n",
     'message_log': [{'lc': 1,
       'type': 'constructor',
       'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'],
       'kwargs': {'content': '',
        'example': False,
        'additional_kwargs': {'function_call': {'name': 'get_prompt_outputs_from_expert_rating',
          'arguments': '{"expert_rating": "5"}'}},
        'tool_call_chunks': [{'name': 'get_prompt_output

In [ ]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
)

In [ ]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[reasoningengine,langchain]",
        "google-cloud-bigquery",
        "requests",
    ],
)

In [ ]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Swedish currency today?"
)

In [ ]:
reasoning_engines.ReasoningEngine.list()